list of techniques [here](https://blog.floydhub.com/ten-techniques-from-fast-ai/)

TODO
- learning rate diff each layer
- freeze layer
- data loader wrapper

In [1]:
%reload_ext autoreload
%autoreload 2
import torch
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchvision import transforms
import torchvision
import copy

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)

In [3]:
transform = transforms.Compose([
#                                    transforms.RandomAffine(15, scale=(.9,1.2)),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5,), (1,))
                               ])
# mnist_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# mnist_testset = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
# data_loader_train = DataLoader(mnist_trainset, 64, shuffle=True, num_workers=8)
# data_loader_test = DataLoader(mnist_testset, 64, shuffle=True, num_workers=8)

In [4]:
temp_raw_data_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
class TempDataSet(Dataset):
    def __getitem__(self, index):
        return temp_raw_data_set[index]
    def __len__(self):
        return 6
temp_data_set = TempDataSet()
data_loader_train = DataLoader(temp_data_set, 3, shuffle=True, num_workers=8)
data_loader_test = DataLoader(temp_data_set, 3, shuffle=True, num_workers=8)

Files already downloaded and verified


In [5]:
import RestNet
model = RestNet.ResNet18().to(device)
criterion = F.cross_entropy

In [6]:
# model = torchvision.models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 10)
# model = model.to(device)

### Learning Rates [Choose one]
1. Different LR [worse] (maybe we have to use log scale)

In [7]:
lr = 0.001
num_children = len([c for c in model.children()])
param_groups = []
for i, child in enumerate(model.children()):
    denorm = num_children-i
    param_groups.append({
        'params': child.parameters(), 'lr': lr/denorm
    })
optimizer = optim.Adam(param_groups, lr=0.001, weight_decay=0.0) #weight_decay (L2 regularization) makes thing worse

In [8]:
for param_group in optimizer.param_groups:
    print(param_group['lr'])

0.00014285714285714287
0.00016666666666666666
0.0002
0.00025
0.0003333333333333333
0.0005
0.001


2. Static Learning Rate

In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Train

In [10]:
from exitai.learner import Learner
learner = Learner(data_loader_train, data_loader_test, model, criterion)

# test
learner.fit(0.003, 50)
# learner.lr_find(1e-4, 1, 2)

---- epoch:0 ------
   [train] Average loss: 86.2075, acc: 16.67%
   [test] Average loss: 74.1362, acc: 33.33%
---- epoch:1 ------
   [train] Average loss: 59.5442, acc: 33.33%
   [test] Average loss: 68.0289, acc: 33.33%
---- epoch:2 ------
   [train] Average loss: 52.6429, acc: 50.00%
   [test] Average loss: 119.5856, acc: 33.33%
---- epoch:3 ------
   [train] Average loss: 30.9985, acc: 66.67%
   [test] Average loss: 114.9487, acc: 33.33%
---- epoch:4 ------
   [train] Average loss: 23.5989, acc: 83.33%
   [test] Average loss: 82.5709, acc: 33.33%
---- epoch:5 ------


Traceback (most recent call last):
  File "/Users/epinyoanun/miniconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/epinyoanun/miniconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/epinyoanun/miniconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/epinyoanun/miniconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/epinyoanun/miniconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/epinyoanun/miniconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/epinyoanun/miniconda3/lib/

KeyboardInterrupt: 